In [143]:
import spacy
import pickle
import random
import json

In [144]:
import json

# Load JSON data from file
with open("data/train_data (2).json", "r") as json_file:
    json_data = json.load(json_file)

# Extract text and annotations from each entry
train_data = []
for entry in json_data:
    text = entry[0]  # Extract text from the first element of the entry list
    annotations = entry[1]["entities"]  # Extract annotations from the "entities" key of the second element
    train_data.append((text, {"entities": annotations}))



In [145]:
print(train_data[0])

('Govardhana K Senior Software Engineer  Bengaluru, Karnataka, Karnataka - Email me on Indeed: indeed.com/r/Govardhana-K/ b2de315d95905b68  Total IT experience 5 Years 6 Months Cloud Lending Solutions INC 4 Month • Salesforce Developer Oracle 5 Years 2 Month • Core Java Developer Languages Core Java, Go Lang Oracle PL-SQL programming, Sales Force Developer with APEX.  Designations & Promotions  Willing to relocate: Anywhere  WORK EXPERIENCE  Senior Software Engineer  Cloud Lending Solutions -  Bangalore, Karnataka -  January 2018 to Present  Present  Senior Consultant  Oracle -  Bangalore, Karnataka -  November 2016 to December 2017  Staff Consultant  Oracle -  Bangalore, Karnataka -  January 2014 to October 2016  Associate Consultant  Oracle -  Bangalore, Karnataka -  November 2012 to December 2013  EDUCATION  B.E in Computer Science Engineering  Adithya Institute of Technology -  Tamil Nadu  September 2008 to June 2012  https://www.indeed.com/r/Govardhana-K/b2de315d95905b68?isid=rex-

In [146]:
nlp = spacy.blank('en')

def train_model(train_data):
    if 'ner' not in nlp.pipe_names:
        ner = nlp.create_pipe('ner')
        nlp.add_pipe(ner, last = True)
    
    for _, annotation in train_data:
        for ent in annotation['entities']:
            ner.add_label(ent[2])
            
    
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
    with nlp.disable_pipes(*other_pipes):  # only train NER
        optimizer = nlp.begin_training()
        for itn in range(10):
            print("Starting iteration " + str(itn))
            random.shuffle(train_data)
            losses = {}
            index = 0
            for text, annotations in train_data:
                try:
                    nlp.update(
                        [text],  # batch of texts
                        [annotations],  # batch of annotations
                        drop=0.2,  # dropout - make it harder to memorise data
                        sgd=optimizer,  # callable to update weights
                        losses=losses)
                except Exception as e:
                    pass
                
            print(losses)



In [147]:
print(annotations)


[[587, 669, 'Skills'], [512, 553, 'College Name'], [503, 510, 'Degree'], [177, 193, 'Designation'], [34, 58, 'Companies worked at'], [14, 30, 'Designation'], [0, 13, 'Name']]


In [148]:
# Train the model
train_model(train_data)

c:\desktop file\job_counselor\venv\lib\site-packages\spacy\language.py:635: UserWarning: [W033] Training a new parser or NER using a model with an empty lexeme normalization table. This may degrade the performance to some degree. If this is intentional or this language doesn't have a normalization table, please ignore this warning.
  proc.begin_training(
c:\desktop file\job_counselor\venv\lib\site-packages\spacy\language.py:635: UserWarning: [W034] Please install the package spacy-lookups-data in order to include the default lexeme normalization table for the language 'en'.
  proc.begin_training(


Statring iteration 0


c:\desktop file\job_counselor\venv\lib\site-packages\spacy\language.py:482: UserWarning: [W030] Some entities could not be aligned in the text "Jacob Philip Kottayam, Kerala - Email me on Indeed..." with entities "[[2165, 2214, 'Skills'], [2145, 2155, 'Degree'], [...". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities (with BILUO tag '-') will be ignored during training.
  gold = GoldParse(doc, **gold)
c:\desktop file\job_counselor\venv\lib\site-packages\spacy\language.py:482: UserWarning: [W030] Some entities could not be aligned in the text "Karthik Gururaj Technical Lead at Infosys Ltd. - P..." with entities "[[2814, 2839, 'Degree'], [2773, 2812, 'Degree'], [...". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities (with BILUO tag '-') will be ignored during training.
  gold = GoldParse(doc, **gold)
c:\desktop file\job_counselor\venv\lib\site-packages\spacy

{'ner': 16774.95193812403}
Statring iteration 1
{'ner': 13120.362489041969}
Statring iteration 2
{'ner': 12132.050194201596}
Statring iteration 3
{'ner': 8951.429815718193}
Statring iteration 4
{'ner': 8913.60806220894}
Statring iteration 5
{'ner': 6208.073826152139}
Statring iteration 6
{'ner': 6310.070250778226}
Statring iteration 7
{'ner': 5457.426921583212}
Statring iteration 8
{'ner': 5135.611555378932}
Statring iteration 9
{'ner': 5398.617797707489}


In [149]:
# Save the model to disk
nlp.to_disk('nlp_model')

In [150]:
# Example of processing resumes
nlp_model = spacy.load('nlp_model')

In [151]:
doc = nlp_model(train_data[0][0])
for ent in doc.ents:
    print(f'{ent.label_.upper():{30}}- {ent.text}')

NAME                          - Gunjan Nayyar
LOCATION                      - Hoshiarpur
EMAIL ADDRESS                 - indeed.com/r/Gunjan-Nayyar/a5819ca6733a0f41
DESIGNATION                   - Presently completed 6 months internship
COMPANIES WORKED AT           - INFOSYS
COMPANIES WORKED AT           - INFOSYS
DESIGNATION                   - Member of Discipline Committee in ALGORYTHM
GRADUATION YEAR               - 2016
COLLEGE NAME                  - in College on National Blood Donation day on 17thNov.2016. • Member of Discipline Committee in ALGORYTHM
GRADUATION YEAR               - 2014
DEGREE                        - B. E. in CSE
COLLEGE NAME                  - Chitkara University
GRADUATION YEAR               - 2018
GRADUATION YEAR               - 2014


In [155]:
import sys, fitz
fname = input()
doc = fitz.open(fname)
text = ""
for page in doc:
    text = text + page.get_text()

tx = " ".join(text.split('\n'))
print(tx)

Education Bachelor of Technology- Artificial Intelligence And Data Science Jabalpur Engineering College 7.81 CGPA 09/2023 – 05/2025 Jabalpur, MP 12th Barfani Academy 92.4 % 04/2020 – 07/2021 Indore, MP Skills Python-Flask Machine learning Data cleaning and preprocessing SQL and Database Management Communication skills Data Analysis and visualization Natural language processing Exploratory Data Analysis (EDA) Statistical Analysis Fast Learner Projects Diamond Price Prediction The Diamond Price Prediction project focuses on leveraging machine learning techniques to predict the market  value of diamonds based on various intrinsic characteristics. The dataset used in this project includes features such  as carat weight, cut, color, clarity, and depth, among others. Diabetes Prediction This project focuses on developing a predictive model for diabetes detection using machine learning algorithms.  The project involves data preprocessing, feature selection, and the implementation of classific

In [156]:
doc = nlp_model(tx)
for ent in doc.ents:
    print(f'{ent.label_.upper():{30}}- {ent.text}')

NAME                          - Education Bachelor
DESIGNATION                   - Barfani Academy 92.4


In [157]:
def extract_skills(text):
    # Convert the text to lowercase for case-insensitive matching
    lowercase_text = text.lower()
    
    # Define keywords that may indicate the start of the skills section
    skill_keywords = ["skills", "technical skills", "proficiency", "expertise", "competencies"]
    
    # Find the index of the keyword that indicates the start of the skills section
    start_index = None
    for keyword in skill_keywords:
        index = lowercase_text.find(keyword)
        if index != -1:
            start_index = index
            break
    
    # If no keyword is found, return None
    if start_index is None:
        return None
    
    # Find the end of the skills section (assuming it ends at the next section)
    end_index = lowercase_text.find("achievements", start_index)
    if end_index == -1:
        end_index = len(text)
    
    # Extract the skills section
    skills_section = text[start_index:end_index].strip()
    
    return skills_section

# Example usage:
resume_text = tx

skills_section = extract_skills(resume_text)
if skills_section:
    print(skills_section)
else:
    print("Skills section not found.")



Skills Python-Flask Machine learning Data cleaning and preprocessing SQL and Database Management Communication skills Data Analysis and visualization Natural language processing Exploratory Data Analysis (EDA) Statistical Analysis Fast Learner Projects Diamond Price Prediction The Diamond Price Prediction project focuses on leveraging machine learning techniques to predict the market  value of diamonds based on various intrinsic characteristics. The dataset used in this project includes features such  as carat weight, cut, color, clarity, and depth, among others. Diabetes Prediction This project focuses on developing a predictive model for diabetes detection using machine learning algorithms.  The project involves data preprocessing, feature selection, and the implementation of classification algorithms to  achieve accurate and reliable predictions. MNIST Digits Classification The MNIST Digit Classification is a fundamental undertaking in the field of computer vision and machine  learn